# Inverse Material Renderer
### CS 77/277: Computer Graphics
### Jessie Li & Michael Riad Zaky

* diffuse results
* solve with first 8 scenes first (0-7, constant lighting position)
* then, add the next 8 scenes (total 0 - 15, same camera angle, different lighting positions)
* see if adding more scenes improves estimates

* set minimum number of scenes to 4 (v. not? gray stuff with artifacts)
* raise number of guesses/iterations
* clamp -- option to turn on and off

In [ ]:
from bsdf import NUM_VAR_BSDF, NUM_VAR_DIFFUSE
import json
import numpy as np
from PIL import Image
import texelSolve
import time
from concurrent.futures import ProcessPoolExecutor

# import importlib
# importlib.reload(texelSolve)

# data
DATA_NAME = 'cerberus512'
DATA_FILE = f'./data/{DATA_NAME}.json'

# optimization
NUM_PROCESSES = 8
PROCESS_TIMEOUT = 3000
METHOD = 'Nelder-Mead'
METHOD_OPTIONS = { 'xatol': 1/256, 'maxiter': 100, 'adaptive': True }
MAX_ITERATIONS = 100

# results
OUT_DIRECTORY = 'out/{DATA_NAME}'
IMAGE_SUFFIX = 'clamped'

IMAGE_PREDICTIONS_RGB = f'{OUT_DIRECTORY}/rgb-{IMAGE_SUFFIX}.png'
IMAGE_PREDICTIONS_ROUGHNESS = f'{OUT_DIRECTORY}/rough-{IMAGE_SUFFIX}.png'
IMAGE_PREDICTIONS_METALLIC = f'{OUT_DIRECTORY}/metal-{IMAGE_SUFFIX}.png'
IMAGE_ERRORS = f'{OUT_DIRECTORY}/errors-{IMAGE_SUFFIX}.png'

IMAGE_RGB_GAMMA = f'{OUT_DIRECTORY}/rgb-gamma-22-{IMAGE_SUFFIX}.png'
IMAGE_ROUGH_GAMMA = f'{OUT_DIRECTORY}/rough-gamma-22-{IMAGE_SUFFIX}.png'
IMAGE_METAL_GAMMA = f'{OUT_DIRECTORY}/metal-gamma-22-{IMAGE_SUFFIX}.png'

GAMMA = 2.2


In [ ]:
def loadJsonData(filePath: str):
    try:
        with open(filePath) as file:
            data = json.load(file)
    except Exception as e:
        print('Error loading JSON:', e)
        return
    
    bsdf = data.get('bsdf', '(null)')
    resolution = data.get('solveTexSize', '(null)')
    
    print(f'File: {filePath}')
    print(f'BSDF: {bsdf}')
    print (f'Resolution: {resolution}')
    print('------------------------------------------------------------')

    return bsdf, resolution, data['column_row_scene']

# -----------------------------------------------------------------
# load data
# -----------------------------------------------------------------
bsdf, resolution, data = loadJsonData(DATA_FILE)
print('\nData loaded.\n')

File: ./data/cerberus512.json
BSDF: bsdf
Resolution: [512, 512]
------------------------------------------------------------

Data loaded.



In [ ]:
# -----------------------------------------------------------------
# solve, multithreaded with ProcessPoolExecuter
# -----------------------------------------------------------------

chunkSize = len(data) // NUM_PROCESSES
dataItems = list(data.items())

errors = np.zeros(resolution) 
predictions = np.zeros((*resolution, NUM_VAR_BSDF))

# ProcessPoolExecutor usage:
# stackoverflow.com/questions/75838200/whats-the-simple-way-to-get-the-return-value-of-a-function-passed-to-multiproce
with ProcessPoolExecutor() as executor:
    start_time = time.time()

    futures = []
    
    for i in range(NUM_PROCESSES):
        chunk = dataItems[i * chunkSize : (i + 1) * chunkSize]
        futures.append(executor.submit(texelSolve.processChunkSeparately, bsdf, chunk, resolution))
    
    for future in futures:
        try:
            chunkPredictions, chunkErrors = future.result(timeout=3000)
            
            # update predictions
            mask = chunkPredictions != 0
            predictions[mask] = chunkPredictions[mask]  
            print('Updated predictions.')
            
            # update errors
            mask = chunkErrors != 0
            errors[mask] = chunkErrors[mask]
            print('Updated errors.')

        except TimeoutError:
            print("Timed out.")
        except Exception as e:
            print(f"Error: {e}")
    
    end_time = time.time()
    print(f"\nTime elapsed ({POOL_SIZE} threads): {end_time - start_time:.6f} seconds")
    print(f'Nonzero count after solve (should be > 0): {np.count_nonzero(predictions)}')

0

In [ ]:
# -----------------------------------------------------------------
# save results
# -----------------------------------------------------------------
print(f'Max error: {errors.max()}')
print(f'Mean error (including zeros): {errors.mean()}')
print(f'Mean error (excluding zeros): {p.mean(errors[np.nonzero(errors)])}')

if (bsdf == 'diffuse'):
    predictedRGB = predictions
    predictionsImage = Image.fromarray((predictedRGB * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_RGB)

if (bsdf == 'bsdf'):
    print(predictions.size)
    predictedRGB = predictions[:, :, :3]
    predictedRoughness = predictions[:, :, 3]
    predictedMetallic = predictions[:, :, 4]
    
    predictionsImage = Image.fromarray((predictedRGB * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_RGB)
    
    predictionsImage = Image.fromarray((predictedRoughness * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_ROUGHNESS)
    
    predictionsImage = Image.fromarray((predictedMetallic * 255).astype(np.uint8))
    predictionsImage.save(IMAGE_PREDICTIONS_METALLIC)

# error image
errorsImage = Image.fromarray((errors/errors.max() * 255).astype(np.uint8))
errorsImage.save(IMAGE_ERRORS)

# gamma correction: https://gist.github.com/rkdgusrn1212/5eb95c0c019e280f07269967017c4f38
im = Image.open(IMAGE_PREDICTIONS_RGB)

row = im.size[0]
col = im.size[1]
resultImage = Image.new("RGB", (row, col))

for x in range(1 , row):
    for y in range(1, col):
        r = min(255, pow(im.getpixel((x,y))[0]/255, (1/GAMMA))*255)
        g = min(255, pow(im.getpixel((x,y))[1]/255, (1/GAMMA))*255)
        b = min(255, pow(im.getpixel((x,y))[2]/255, (1/GAMMA))*255)
        resultImage.putpixel((x,y), (int(r), int(g), int(b)))
            
resultImage.save(IMAGE_RGB_GAMMA)

if (bsdf == 'bsdf'):
    images = [(IMAGE_PREDICTIONS_METALLIC, IMAGE_METAL_GAMMA),
              (IMAGE_PREDICTIONS_ROUGHNESS, IMAGE_ROUGH_GAMMA)]
    
    for im, outImage in images:
        row = im.size[0]
        col = im.size[1]
        resultImage = Image.new("L", (row, col))

        for x in range(1 , row):
            for y in range(1, col):
                r = min(255, pow(im.getpixel((x,y))/255, (1/GAMMA))*255)
                resultImage.putpixel((x,y), int(r))
            
        resultImage.save(outImage)

1310720
